In [ ]:
#get an abstract
import requests

#capitalization matters
#can enter any wikipedia title here
#subject = 'Conway Twitty'
#emperor penguin
#java
#java (programming language)
#python (programming language)
#python
#Pythonidae
#History of Python
subject = input("What is the subject?")


url = 'https://en.wikipedia.org/w/api.php'
params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }
 
response = requests.get(url, params=params)
data = response.json()
 
page = next(iter(data['query']['pages'].values()))
#print(page)
#Show only the abstract
print(page['extract'][:])


In [ ]:
#get whole text 
import requests
from lxml import html
 
#subject = 'Python (programming language)'
subject = input("What is the subject?")
url = 'https://en.wikipedia.org/w/api.php'
params = {
        'action': 'parse',
        'format': 'json',
        'page': subject,
        'prop': 'text',
        'redirects':''
    }
  
response = requests.get(url, params=params).json()
raw_html = response['parse']['text']['*']
#print(raw_html)
document = html.document_fromstring(raw_html)
#print(document) 
text = ''
for p in document.xpath('//p'):
   text += p.text_content() + '\n'
print(text)

In [ ]:
#make a search
#find occurrences and check if an entity exists and what is the subject name
import requests
 
#query = 'python'
#not case sensitive
query = input("What is the query?") 
url = 'https://en.wikipedia.org/w/api.php'
params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'utf8':1,
            'srsearch':query
        }
 
data = requests.get(url, params=params).json()
#print(data) 
for i in data['query']['search']:
    print(i['title'], ' - Word count: ', i['wordcount'])

In [ ]:
import requests
from bs4 import BeautifulSoup
 
subject = input("What is the subject?") 
 
url = 'https://en.wikipedia.org/w/api.php'
params = {
            'action': 'parse',
            'page': subject,
            'format': 'json',
            'prop':'text',
            'redirects':''
        }
 
response = requests.get(url, params=params)
data = response.json()
 
raw_html = data['parse']['text']['*']
soup = BeautifulSoup(raw_html,'html.parser')
soup.find_all('p') #find all paragraphs
text = ''
 
for p in soup.find_all('p'):
    text += p.text
 
print(text[:60])
print('Text length: ', len(text))

In [ ]:
#find all links on the page
#get titles and count
import requests
 
subject = input('What is the subject?')
 
url = 'https://en.wikipedia.org/w/api.php'
 
params = {
    'action': 'query',
    'format': 'json',
    'titles': subject,
    'prop': 'links',
    'pllimit': 'max',
    'redirects':''
}
 
response = requests.get(url=url, params=params)
data = response.json()
 
pages = data['query']['pages']
page = 1
page_titles = []
 
for key, val in pages.items():
    for link in val['links']:
        page_titles.append(link['title'])
 
while 'continue' in data:
    plcontinue = data['continue']['plcontinue']
    params['plcontinue'] = plcontinue
 
    response = requests.get(url=url, params=params)
    data = response.json()
    pages = data['query']['pages']
 
    page += 1
 
    for key, val in pages.items():
        for link in val['links']:
            page_titles.append(link['title'])
 
print(page_titles[:3])
print('number of links: ',len(page_titles))

In [ ]:
#fix with pandas
from bs4 import BeautifulSoup
import pandas as pd
import requests
 
subject = input('What subject?')#'Machine Learning'
 
url = 'https://en.wikipedia.org/w/api.php'
params = {
            'action':'parse',
            'prop':'text',
            'format':'json',
            'page':subject,
            'section':0,
            'redirects':''
        }
data = requests.get(url, params=params).json()
 
soup = BeautifulSoup(data['parse']['text']['*'],'html.parser')
 
infobox = soup.find('table',{'class':'infobox','class':'sidebar'})
 
a_tags = infobox.find_all('a', href=True)
 
links = []
for tag in a_tags:
    if not tag.text == '' and 'wiki' in tag['href']:
        links.append({'anchor':tag.text, 'href':tag['href']})
 
pd.DataFrame(links).head()

In [ ]:
import requests
from lxml import html
url = 'https://en.wikipedia.org/w/api.php'

##Clarify the one we need
subject=input('What subject do you want information on?')
#1. check the wiki name
#make a search
#find occurrences and check if an entity exists and what is the subject name
params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'utf8':1,
            'srsearch':subject
        }
 
data = requests.get(url, params=params).json()
#print(len(data['query']['search']))
subjectConfirmed=0
count=0
if len(data['query']['search'])> 1:
    for i in data['query']['search']:
        print('CHOICE',count,i['title'], ' - Word count: ', i['wordcount'])
        count=count+1
    subjectConfirmed=input('Please enter the number of the topic you were requesting')
    subject=data['query']['search'][int(subjectConfirmed)]['title']
#print(subjectConfirmed)    
#print(data['query']['search'][int(subjectConfirmed)]['title'])

#2. grab the document
params = {
        'action': 'parse',
        'format': 'json',
        'page': subject,
        'prop': 'text',
        'redirects':''
    }
  
response = requests.get(url, params=params).json()
raw_html = response['parse']['text']['*']
#print(raw_html)
document = html.document_fromstring(raw_html)
#print(document) 
text = ''
for p in document.xpath('//p'):
   text += p.text_content() + '\n'
#print(len(text))
#print(text[:50])

#3. now we can store this data, process it, etc.
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords #stopword handling

english_stops = set(stopwords.words('english'))

words=word_tokenize(text) 
sentences=sent_tokenize(text)
#TODO: Decide on a tokenizer
print('# words',len(words))#has commas, ( etc.
print('# sentences',len(sentences))
withoutStops=[word for word in words if word not in english_stops]
print('# words without stop words',len(withoutStops))